# Prediction Insights

## Overview

Howso Engine enables powerful predictions with complete attribution and detailed explanations to make learning from
and debugging your data and predictions as easy as possible. For more information on predictions with Howso Engine,
check out the [predictions user guide](https://docs.howso.com/user_guide/basic_capabilities/predictions.html).

In [1]:
from pprint import pprint

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Setup

This recipe will focus on the insights gained from predictions. Specifically, insights from:

- Influential Cases,
- Feature Contributions,
- Residual and Similarity Conviction,
- and Categorical Action Probabilities.

The [basic workflow guide](https://docs.howso.com/user_guide/basic_capabilities/basic_workflow.html) covers creating a Trainee and performing a basic react in detail.

### Load Data and Create Trainee

In [2]:
df = pd.read_csv("../../data/iris/iris.tsv.gz", sep="\t", compression="gzip")
train_data = df.iloc[:-30]
new_data = df[~df.index.isin(train_data.index)]
features = infer_feature_attributes(train_data)

t = Trainee(features=features)

df

Version 28.1.0 of Howso Engine™ is available. You are using version 28.0.4.dev3+g0840d31.
The following parameters from your configuration will override the default Amalgam parameters: {'library_path'}


,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,27,5.2,3.5,1.5,0.2,Iris-setosa
1,14,5.8,4.0,1.2,0.2,Iris-setosa
2,25,5.0,3.0,1.6,0.2,Iris-setosa
3,82,5.8,2.7,3.9,1.2,Iris-versicolor
4,97,6.2,2.9,4.3,1.3,Iris-versicolor
...,...,...,...,...,...,...
145,126,6.2,2.8,4.8,1.8,Iris-virginica
146,124,6.7,3.3,5.7,2.1,Iris-virginica
147,78,6.0,2.9,4.5,1.5,Iris-versicolor
148,125,7.2,3.2,6.0,1.8,Iris-virginica


### Train, Analyze, and React

In [3]:
t.train(train_data)
t.analyze()
t.react_into_features(similarity_conviction=True)

reaction = t.react(
    contexts=new_data,
    context_features=["sepal length", "sepal width", "petal length", "petal width"],
    action_features=["class"],
    details={
        "influential_cases": True,
        "similarity_conviction": True,
        "feature_contributions_robust": True,
        "feature_residuals_robust": True,
        "case_feature_residual_convictions_robust": True,
        "categorical_action_probabilities": True,
    }
)

Note that, unlike in the basic workflow guide, we include several `details` in the react call and predict multiple cases as opposed to one. These are what will enable the
insights that we're going to get after the predictions are made.

For more information, see [the API documentation for `Trainee.react()`](https://docs.howso.com/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

### Inspect the Predictions

Howso Engine has high accuracy even on small datasets.

Here we see a comparison of the trained data (dots) and the predicted data (stars). This plot shows that the predictions align well with the trained data.

In [4]:
train_data = train_data.astype({"class": str})
predicted_data = pd.concat([new_data.reset_index(drop=True).drop(columns="class"), reaction["action"]], axis=1)
predicted_data = predicted_data.astype({"class": str})

cmap = px.colors.qualitative.D3
fig = go.Figure()
for i, (label, group) in enumerate(train_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        name=label,
        marker=dict(color=cmap[i], opacity=0.75),
        legendgroup="trained",
        legendgrouptitle_text="Trained class",
    ))

for i, (label, group) in enumerate(predicted_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        marker=dict(size=12, symbol="star", color=cmap[i], opacity=0.75),
        name=label,
        legendgroup="predicted",
        legendgrouptitle_text="Predicted class",
        hovertext=group.index,
    ))

fig.update_layout(
    xaxis_title="Petal Length",
    yaxis_title="Petal Width",
    width=1250,
    title="Trained and Predicted Values"
)
fig.show()

For categorical action features, the prediction can be further understood with the `categorical_action_probabilities` detail.  
This shows the probability of each possible value of the target feature. This information can highlight cases that are on the
border of two classes, like some of the above points are.  The closer a case gets to a class border, the more mixed the
categorical action probabilities may get.

In [5]:
pprint(reaction["details"]["categorical_action_probabilities"], compact=True)

[{'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-versicolor': 0.2595877181054522,
            'Iris-virginica': 0.7404122818945479}},
 {'class': {'Iris-versicolor': 0.10797425100734902,
            'Iris-virginica': 0.8920257489926509}},
 {'class': {'Iris-versicolor': 0.22567888149992943,
            'Iris-virginica': 0.7743211185000707}},
 {'class': {'Iris-versicolor': 0.5287044940312792,
            'Iris-virginica': 0.47129550596872083}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.1589482692839984,
            'Iris-virginica': 0.8410517307160016}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-virginica': 1}}, {'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.2803732577088749,
            'Iris-virginica': 0.7196267422911251}},
 {'class': {'Iris-versicolor': 0.9324674426706622,
            'Iris-virginica': 0.06753255732933786}},

For example, if we look at case `4`, we see a mix of probabilities because this case is on the border of two classes.

In [6]:
reaction["details"]["categorical_action_probabilities"][4]

{'class': {'Iris-versicolor': 0.5287044940312792,
  'Iris-virginica': 0.47129550596872083}}

### Insight 1: Which Cases Contributed?

Howso provides complete attribution for any and all predictions, showing exactly which cases influenced each prediction.  This can be used to understand or debug predictions.  For instance,
we can inspect the influential cases for one of the cases (case `4`) that was on the decision boundary in the plot above.  We can see that there is a mix of class values and no cases stand
out in terms of influence weight:

In [7]:
inf_cases_4 = pd.DataFrame(reaction["details"]["influential_cases"][4])
inf_cases_4

,sepal length,petal width,petal length,sepal width,.session_training_index,.session,.influence_weight,class
0,6.1,1.4,4.7,2.9,89,02da825f-fb1c-4015-9a53-b9e876013a4b,0.131993,Iris-versicolor
1,6.1,1.4,4.6,3.0,26,02da825f-fb1c-4015-9a53-b9e876013a4b,0.110173,Iris-versicolor
2,6.3,1.8,5.6,2.9,61,02da825f-fb1c-4015-9a53-b9e876013a4b,0.087547,Iris-virginica
3,6.1,1.2,4.7,2.8,100,02da825f-fb1c-4015-9a53-b9e876013a4b,0.076784,Iris-versicolor
4,6.3,1.5,4.9,2.5,107,02da825f-fb1c-4015-9a53-b9e876013a4b,0.076571,Iris-versicolor
5,6.3,1.5,5.1,2.8,99,02da825f-fb1c-4015-9a53-b9e876013a4b,0.074902,Iris-virginica
6,6.4,2.1,5.6,2.8,36,02da825f-fb1c-4015-9a53-b9e876013a4b,0.068016,Iris-virginica
7,6.1,1.3,4.0,2.8,30,02da825f-fb1c-4015-9a53-b9e876013a4b,0.067571,Iris-versicolor
8,6.8,1.4,4.8,2.8,57,02da825f-fb1c-4015-9a53-b9e876013a4b,0.065613,Iris-versicolor
9,6.0,1.5,5.0,2.2,34,02da825f-fb1c-4015-9a53-b9e876013a4b,0.065126,Iris-virginica


Compare that to the influential cases for case `20`, which is firmly in the center of the solo cluster.  Here we see a single class value.

In [8]:
inf_cases_20 = pd.DataFrame(reaction["details"]["influential_cases"][20])
inf_cases_20

,sepal length,petal width,petal length,sepal width,.session_training_index,.session,.influence_weight,class
0,5.0,0.4,1.6,3.4,63,02da825f-fb1c-4015-9a53-b9e876013a4b,0.136554,Iris-setosa
1,5.4,0.2,1.7,3.4,9,02da825f-fb1c-4015-9a53-b9e876013a4b,0.109539,Iris-setosa
2,5.1,0.2,1.5,3.4,85,02da825f-fb1c-4015-9a53-b9e876013a4b,0.091503,Iris-setosa
3,5.1,0.4,1.5,3.7,117,02da825f-fb1c-4015-9a53-b9e876013a4b,0.087319,Iris-setosa
4,5.0,0.2,1.5,3.4,69,02da825f-fb1c-4015-9a53-b9e876013a4b,0.079465,Iris-setosa
5,5.4,0.2,1.5,3.7,104,02da825f-fb1c-4015-9a53-b9e876013a4b,0.078597,Iris-setosa
6,5.4,0.4,1.3,3.9,21,02da825f-fb1c-4015-9a53-b9e876013a4b,0.075887,Iris-setosa
7,5.4,0.4,1.7,3.9,47,02da825f-fb1c-4015-9a53-b9e876013a4b,0.075887,Iris-setosa
8,5.2,0.2,1.4,3.4,43,02da825f-fb1c-4015-9a53-b9e876013a4b,0.064999,Iris-setosa
9,5.2,0.2,1.5,3.5,0,02da825f-fb1c-4015-9a53-b9e876013a4b,0.060321,Iris-setosa


With the influential cases we can derive additional insights, such as identifying anomalous cases from within the influential cases using `similarity_conviction`.  
This can help to identify data that are making predictions more noisy or identify what type of data should be collected to improve predictive power in the future.

In [9]:
inf_case_indices = inf_cases_4[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"]
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
9,6.0,2.2,5.0,1.5,Iris-virginica,0.539232
4,6.3,2.5,4.9,1.5,Iris-versicolor,0.792197
5,6.3,2.8,5.1,1.5,Iris-virginica,0.898919
8,6.8,2.8,4.8,1.4,Iris-versicolor,0.979917
11,6.4,2.7,5.3,1.9,Iris-virginica,1.053634
1,6.1,3.0,4.6,1.4,Iris-versicolor,1.061953
3,6.1,2.8,4.7,1.2,Iris-versicolor,1.098214
2,6.3,2.9,5.6,1.8,Iris-virginica,1.116684
12,6.1,3.0,4.9,1.8,Iris-virginica,1.157790
7,6.1,2.8,4.0,1.3,Iris-versicolor,1.168651


In [10]:
inf_case_indices = inf_cases_20[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"],
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
7,5.4,3.9,1.7,0.4,Iris-setosa,0.789016
6,5.4,3.9,1.3,0.4,Iris-setosa,0.862923
11,4.6,3.4,1.4,0.3,Iris-setosa,0.909119
10,5.3,3.7,1.5,0.2,Iris-setosa,0.924560
0,5.0,3.4,1.6,0.4,Iris-setosa,0.966282
1,5.4,3.4,1.7,0.2,Iris-setosa,1.058545
3,5.1,3.7,1.5,0.4,Iris-setosa,1.096828
5,5.4,3.7,1.5,0.2,Iris-setosa,1.127392
2,5.1,3.4,1.5,0.2,Iris-setosa,1.259162
12,5.1,3.5,1.4,0.3,Iris-setosa,1.334723


### Insight 2: Which Features Contributed?

In addition to providing attribution to cases, Howso also provides robust feature contributions to explain which features contributed to each prediction.
When inspecting the feature importances for case `13`,  we see that `petal width` and `petal length` provided the majority of the contribution to the 
prediction, whereas for case `20` `petal width` and `petal length` only have slightly higher contributions than the other features.  This could indicate 
that different features are more important for predicting different classes within this dataset and that focusing on those features would be prudent.

Identifying that certain cases that influenced a prediction are anomalous can help to explain low predictive power in certain parts of the data or other
issues that may appear during deployment.

In [11]:
fcs_13 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][13:14], index=[13])
fcs_20 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][20:21], index=[20])

display(fcs_13)
display(fcs_20)

,sepal width,sepal length,petal width,petal length
13,0.054837,0.128141,0.308447,0.298769


,sepal width,sepal length,petal width,petal length
20,0.118496,0.118917,0.158989,0.153181


### Insight 3: How Certain is the Prediction?

[Residuals](https://docs.howso.com/user_guide/basic_capabilities/residuals.html) can characterize the uncertainty of the data around the prediction.  This will tell us which features are hard to predict in the region of the data around each case we're predicting.
If a prediction is less accurate than expected, this can explain which features were noisy and may have contributed to the problem.

In [12]:
pd.DataFrame(reaction["details"]["feature_residuals_robust"])

,sepal length,petal width,petal length,sepal width,class
0,0.403728,0.169559,0.323572,0.199032,0.266149
1,0.372697,0.268837,0.461490,0.232683,0.365244
2,0.437603,0.339065,0.540012,0.160669,0.238121
3,0.380100,0.268216,0.434263,0.226865,0.346723
4,0.345067,0.201421,0.396560,0.250438,0.267561
5,0.339010,0.131704,0.332973,0.152123,0.124014
6,0.312111,0.185495,0.381352,0.228970,0.082744
7,0.385033,0.300295,0.452539,0.155574,0.271013
8,0.332298,0.158037,0.341377,0.203010,0.224229
9,0.308422,0.141852,0.348138,0.169970,0.122461


We can also use [`residual conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#prediction-residual-conviction) to determine which features
are uncertain in a scale-invariant manner, which can be useful if you wish to compare different features of different scales against each other.  Higher residual conviction
indicates something that is less surprising than expected and lower redsidual conviction indicates the opposite.

In [13]:
pd.DataFrame(reaction["details"]["case_feature_residual_convictions_robust"])

,sepal length,petal width,petal length,sepal width,class
0,1.127942,1.134861,0.966987,1.991696,0.984665
1,1.864496,1.162410,1.267351,0.743423,0.904140
2,1.162034,0.742276,0.905198,2.064518,1.042898
3,0.647946,0.930452,0.856988,0.515133,0.872205
4,1.653969,0.701745,0.314707,0.756120,0.777068
5,1.247961,2.268054,1.704988,0.863904,0.840815
6,1.098038,3.135647,1.105812,0.736975,0.728849
7,0.968958,1.188002,1.061948,0.835122,1.152357
8,0.795975,0.779004,0.698871,0.336850,1.166637
9,1.222966,0.979704,1.327802,0.460838,1.723441


### Insight 4: How Anomalous are the Predicted Cases?

By getting the [`similarity conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#similarity-conviction) of the cases that we predict, 
we can determine which of them are anomalous relative to the trained data. This could help to highlight cases that are unusually difficult or easy to predict, such as can be the case
when model drift occurs as a form of model monitoring.

In [14]:
pprint(reaction["details"]["similarity_conviction"], compact=True)

[2.457885693293746, 1.5354014069751163, 1.9702417257436509, 1.1930428026510889,
 0.9049949791719423, 2.644985108390524, 2.3186690612841194, 1.7631066340379993,
 1.1947441984854577, 2.4719646523918306, 3.3614233841581114, 2.921636828632703,
 4.439559676581496, 1.6872697549679847, 1.6868491390495826, 1.914699978141316,
 1.8491204381234474, 2.9714859744896422, 1.0422352275446207, 0.8457073281021638,
 1.8787051436012008, 2.2917607347470694, 1.0551141322845465, 2.9018060878979877,
 3.169837103507889, 2.084071720024441, 2.005408873499514, 2.308758287437608,
 1.3950592277059894, 1.2402940584624549]


In this case, there are no cases with a low similarity_conviction which indicates that there are no cases that we reacted to which are particularly
anomalous.